# Lending Club Data

## Data Analysis

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from custom_log_regression import CustomLogRegression

DATA_PATH = './data/DR_Demo_Lending_Club_reduced.csv'
RND_STATE = 42

data = pd.read_csv(DATA_PATH, index_col='Id', na_values=['NONE', 'na'])
print(f'Initial data size {data.shape}')

Initial data size (10000, 23)


In [2]:
from category_encoders.one_hot import OneHotEncoder

ModuleNotFoundError: No module named 'category_encoders'

In [36]:
numeric_col = set(data._get_numeric_data().columns)
categorical_col = set(data.columns) - numeric_col

categorical_col

{'addr_state',
 'home_ownership',
 'initial_list_status',
 'policy_code',
 'purpose_cat',
 'pymnt_plan',
 'verification_status',
 'zip_code'}

In [46]:
for col in categorical_col:
    if len(data[col].unique()) == 2:
        print(col)

pymnt_plan
initial_list_status


In [25]:
data._get_numeric_data().columns

Index(['Id', 'is_bad', 'emp_length', 'annual_inc', 'debt_to_income',
       'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog'],
      dtype='object')

In [2]:
numeric_cols = ['annual_inc', 'collections_12_mths_ex_med', 'debt_to_income',
    'delinq_2yrs', 'revol_util', 'emp_length', 'total_acc', 'inq_last_6mths',
    'mths_since_last_major_derog', 'open_acc', 'pub_rec', 'revol_bal']

categorical_cols = ['addr_state', 'home_ownership', 'initial_list_status', 'zip_code', 'pymnt_plan',
    'policy_code', 'verification_status', 'purpose_cat']

In [31]:
# which categorical variables do we have and there values
import numpy as np

for cat in categorical_cols:
    unique_values = np.unique(data[cat].values)
    print(f'{cat} values: {unique_values}')

addr_state values: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN'
 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


TypeError: '<' not supported between instances of 'str' and 'float'

In [4]:
# map selected categorical variables to numerical values
def verification_status_mapper(vstatus):
    if 'VERIFIED' in vstatus:
        return 1
    return 0

def pymnt_plan_mapper(pymnt_plan):
    if pymnt_plan == 'y':
        return 1
    return 0

def initial_list_status_mapper(initial_list_status):
    if initial_list_status == 'm':
        return 1
    return 0

home_ownership_codes = {
        'NONE': None,
        'OTHER': 0,
        'MORTGAGE': 1,
        'OWN': 2,
        'RENT': 3
    }
    
data.verification_status = data.verification_status.map(verification_status_mapper)
data.pymnt_plan = data.pymnt_plan.map(pymnt_plan_mapper)
data.initial_list_status = data.initial_list_status.map(initial_list_status_mapper)
data.policy_code = data.policy_code.map(lambda p: int(p[-1]))
data.home_ownership = data.home_ownership.map(lambda c: home_ownership_codes[c])

In [48]:
data.isna().sum()

is_bad                            0
emp_length                      250
home_ownership                    1
annual_inc                        1
verification_status               0
pymnt_plan                        0
purpose_cat                       0
zip_code                          0
addr_state                        0
debt_to_income                    0
delinq_2yrs                       5
inq_last_6mths                    5
mths_since_last_delinq         6316
mths_since_last_record         9160
open_acc                          5
pub_rec                           5
revol_bal                         0
revol_util                       26
total_acc                         5
initial_list_status               0
collections_12_mths_ex_med       32
mths_since_last_major_derog       0
policy_code                       0
dtype: int64

In [53]:
data_len = len(data)
nas = data.isna().sum()
nas[nas < 0.1 * data_len].index

Index(['is_bad', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'pymnt_plan', 'purpose_cat', 'zip_code',
       'addr_state', 'debt_to_income', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code'],
      dtype='object')

We should drop *mths_since_last_record* and *mths_since_last_delinq*, there are too many N/A values.

In [55]:
'addr_state' not in numeric_cols

True

In [6]:
drop_cols = ['addr_state', 'zip_code', 'purpose_cat', 
             'mths_since_last_delinq', 'mths_since_last_record']
data = data.drop(drop_cols, axis=1)

## Train Classifier

In [7]:
# clean non-numeric values in numeric columns
# not correctly placed
data = data.loc[data.loc[:, 'emp_length'].str.isnumeric().values]
data = data.dropna()

print(f'Data size {data.shape}')

X = data.loc[:, numeric_cols]
y = data.is_bad.values

test_size = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, random_state=RND_STATE)

clf = CustomLogRegression(random_state=RND_STATE)
params = clf.tune_parameters(X_train, y_train)
print(params)

clf.fit(X_train, y_train)
metrics = clf.evaluate(X_test, y_test)

print('Accuracy {:.2}, F1 Score {:.2}, Logistic Loss {:.2}'.format(
    *metrics.values()
))

Data size (9700, 19)


/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mn

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/en

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mn

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/pytho

...

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/pytho

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mn

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/pytho

{'C': 0.0006579332246575682, 'scores': {'accuracy': array([0.50735379, 0.50735379, 0.50735379, 0.50735379, 0.50735379,
       0.50735379, 0.50735379, 0.50735379, 0.50735379, 0.50735379,
       0.50735379, 0.50720652, 0.50705914, 0.50705914, 0.50705914,
       0.5072063 , 0.50735369, 0.50750096, 0.50764824, 0.51795069,
       0.54019058, 0.53724616, 0.54756032, 0.54682438, 0.5480015 ,
       0.5478539 , 0.54888548, 0.54932709, 0.54932709, 0.54844387,
       0.54991651, 0.55094744, 0.55227281, 0.55271453, 0.55300854,
       0.5552167 , 0.55609981, 0.5575718 , 0.55801406, 0.56066339,
       0.55977942, 0.56183997, 0.56154477, 0.56154575, 0.56139858,
       0.56243027, 0.56375574, 0.56375574, 0.56419811, 0.56552305,
       0.56596488, 0.5652285 , 0.56552403, 0.56508253, 0.5652298 ,
       0.56493633, 0.5653786 , 0.5653787 , 0.5653787 , 0.56478971,
       0.56537903, 0.56537914, 0.56537935, 0.56552652, 0.56537914,
       0.56537892, 0.56537914, 0.56537935, 0.56537935, 0.56537935,
       0.5

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/Projects/home-test-dr/custom_log_regression.py:31: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  X_scaled = scaler.transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/

In [16]:
pd.read_csv??